In [ ]:
# import all necessary libraries
import numpy as np
import pandas as pd
import datetime
import torch
import torch.utils
from datetime import timezone
import time

In [ ]:
# import loader, model and utils
from github_dataloader import *
from DyRep import *
from utils import *

In [ ]:
datdir = '/Github'
train_set = GithubDataset('train', data_dir=datdir)
test_set = GithubDataset('test', data_dir=datdir)

# Preview the first few lines of the train set and test set
print("Train set preview (first 5 events):")
for event in train_set.all_events[:5]:
    print(event)

print("\nTest set preview (first 5 events):")
for event in test_set.all_events[:5]:
    print(event)

In [ ]:
# initialize the model
initial_embeddings = np.random.randn(train_set.N_nodes, 32)
A_initial = train_set.get_Adjacency()[0]

In [ ]:
from torch.utils.data import DataLoader
# Create DataLoader for train and test sets
train_loader = DataLoader(train_set, batch_size=200, shuffle=False)
test_loader = DataLoader(test_set, batch_size=200, shuffle=False)

In [ ]:
# initialise the A and z 
N_nodes = A_initial.shape[0]
if A_initial is not None and len(A_initial.shape) == 2:
    A_initial = A_initial[:, :, None]
n_assoc_types,n_event_types = 1, 3
n_relations = n_assoc_types + n_event_types

Adj_all = train_set.get_Adjacency()[0]

if not isinstance(Adj_all, list):
    Adj_all = [Adj_all]

node_degree_global = []
for rel, A in enumerate(Adj_all):
    node_degree_global.append(np.zeros(A.shape[0]))
    for u in range(A.shape[0]):
        node_degree_global[rel][u] = np.sum(A[u])

Adj_all = Adj_all[0]


In [ ]:
# Instantiate the model
model = DyRep_update(
    node_embeddings=initial_embeddings,
    A_initial=A_initial,
    N_surv_samples=5,
    n_hidden=32,
    node_degree_global= node_degree_global,
    N_hops=2,
)

# #   model = DyRep_update(
#     node_embeddings=initial_embeddings,
#     A_initial=A_initial,
#     N_surv_samples=5,
#     n_hidden=32,
#     node_degree_global= node_degree_global,
#     N_hops=1,
# )

In [ ]:
def test(model, n_test_batches=10, epoch=0):
    model.eval()
    loss = 0
    losses =[ [np.Inf, 0], [np.Inf, 0] ]
    n_samples = 0
    # Time slots with 10 days intervals as in the DyRep paper
    timeslots = [t.toordinal() for t in test_loader.dataset.TEST_TIMESLOTS]
    event_types = list(test_loader.dataset.event_types_num.keys()) #['comm', 'assoc']
    # sort it by k
    for event_t in test_loader.dataset.event_types_num:
        event_types[test_loader.dataset.event_types_num[event_t]] = event_t

    event_types += ['Com']

    mar, hits_10 = {}, {}
    for event_t in event_types:
        mar[event_t] = []
        hits_10[event_t] = []
        for c, slot in enumerate(timeslots):
            mar[event_t].append([])
            hits_10[event_t].append([])


    start = time.time()
    with torch.no_grad():
        
        import datetime
        #from datetime import datetime, timezone 
        for batch_idx, data in enumerate(test_loader):
            data[2] = data[2].float()
            data[4] = data[4].double()
            data[5] = data[5].double()
            output = model(data)
            loss += (-torch.sum(torch.log(output[0]) + 1e-10) + torch.sum(output[1])).item()
            for i in range(len(losses)):
                m1 = output[i].min()
                m2 = output[i].max()
                if m1 < losses[i][0]:
                    losses[i][0] = m1
                if m2 > losses[i][1]:
                    losses[i][1] = m2
            n_samples += 1
            A_pred, Survival_term = output[2]
            u, v, k = data[0], data[1], data[3]

            time_cur = data[5]
            m, h = MAR(A_pred, u, v, k, Survival_term=Survival_term)
            assert len(time_cur) == len(m) == len(h) == len(k)
            for t, m, h, k_ in zip(time_cur, m, h, k):
                d = datetime.datetime.fromtimestamp(t.item()).toordinal()
                event_t = event_types[k_.item()]
                for c, slot in enumerate(timeslots):
                    if d <= slot:
                        mar[event_t][c].append(m)
                        hits_10[event_t][c].append(h)
                        if k_ > 0:
                            mar['Com'][c].append(m)
                            hits_10['Com'][c].append(h)
                        if c > 0:
                            assert slot > timeslots[c-1] and d > timeslots[c-1], (d, slot, timeslots[c-1])
                        break

            if batch_idx % 10 == 0:
                print('test', batch_idx)

            if n_test_batches is not None and batch_idx >= n_test_batches - 1:
                break

    time_iter = time.time() - start

    print('\nTEST batch={}/{}, loss={:.3f}, psi={}, loss1 min/max={:.4f}/{:.4f}, '
          'loss2 min/max={:.4f}/{:.4f}, integral time stamps={}, sec/iter={:.4f}'.
          format(batch_idx + 1, len(test_loader), (loss / n_samples),
                 [model.psi[c].item() for c in range(len(model.psi))],
                 losses[0][0], losses[0][1], losses[1][0], losses[1][1],
                 len(model.Lambda_dict), time_iter / (batch_idx + 1)))

    # Report results for different time slots in the test set
    for c, slot in enumerate(timeslots):
        s = 'Slot {}: '.format(c)
        for event_t in event_types:
            sfx = '' if event_t == event_types[-1] else ', '
            if len(mar[event_t][c]) > 0:
                s += '{} ({} events): MAR={:.2f}+-{:.2f}, HITS_10={:.3f}+-{:.3f}'.\
                    format(event_t, len(mar[event_t][c]), np.mean(mar[event_t][c]), np.std(mar[event_t][c]),
                            np.mean(hits_10[event_t][c]), np.std(hits_10[event_t][c]))
            else:
                s += '{} (no events)'.format(event_t)
            s += sfx
        print(s)

    mar_all, hits_10_all = {}, {}
    for event_t in event_types:
        mar_all[event_t] = []
        hits_10_all[event_t] = []
        for c, slot in enumerate(timeslots):
            mar_all[event_t].extend(mar[event_t][c])
            hits_10_all[event_t].extend(hits_10[event_t][c])

    s = 'Epoch {}: results per event type for all test time slots: \n'.format(epoch)
    print(''.join(['-']*100))
    for event_t in event_types:
        if len(mar_all[event_t]) > 0:
            s += '====== {:10s}\t ({:7s} events): \tMAR={:.2f}+-{:.2f}\t HITS_10={:.3f}+-{:.3f}'.\
                format(event_t, str(len(mar_all[event_t])), np.mean(mar_all[event_t]), np.std(mar_all[event_t]),
                       np.mean(hits_10_all[event_t]), np.std(hits_10_all[event_t]))
        else:
            s += '====== {:10s}\t (no events)'.format(event_t)
        if event_t != event_types[-1]:
            s += '\n'
    print(s)
    print(''.join(['-'] * 100))

    return mar_all, hits_10_all, loss / n_samples

In [ ]:
params_main = []
for name, param in model.named_parameters():
    if param.requires_grad:
        params_main.append(param)

In [ ]:
optimizer = torch.optim.Adam([{"params": params_main, "weight_decay":0}], lr=0.0002, betas=(0.5, 0.999))
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, '10', gamma=0.5)

In [ ]:
time_bar = np.zeros((N_nodes, 1)) + train_set.FIRST_DATE.timestamp()

In [ ]:
import copy
def get_temporal_variables():
    variables = {}
    variables['time_bar'] = copy.deepcopy(time_bar)
    variables['node_degree_global'] = copy.deepcopy(node_degree_global)
    variables['time_keys'] = copy.deepcopy(model.time_keys)
    variables['z'] = model.z.clone()
    variables['S'] = model.S.clone()
    variables['A'] = model.A.clone()
    variables['Lambda_dict'] = model.Lambda_dict.clone()
    return variables

In [ ]:
def set_temporal_variables(variables, model, train_loader, test_loader):
    time_bar = copy.deepcopy(variables['time_bar'])
    train_loader.dataset.time_bar = time_bar
    test_loader.dataset.time_bar = time_bar
    model.node_degree_global = copy.deepcopy(variables['node_degree_global'])
    model.time_keys = copy.deepcopy(variables['time_keys'])
    model.z = variables['z'].clone()
    model.S = variables['S'].clone()
    model.A = variables['A'].clone()
    model.Lambda_dict = variables['Lambda_dict'].clone()
    return time_bar

In [ ]:
# initialize the model
epoch_start = 1
epochs = 100
batch_start = 0
batch_size = 200
weight = 1
log_interval = 20
loss_l,losses_events, losses_nonevents, losses_sum = [], [], [], []
test_MAR, test_HITS10, test_loss = [], [], []

In [ ]:
def initalize_state(dataset, keepS=False):
    '''Initializes node embeddings and the graph to the original state after every epoch'''
    Adj_all = dataset.get_Adjacency()[0]

    if not isinstance(Adj_all, list):
        Adj_all = [Adj_all]

    node_degree_global = []
    for rel, A in enumerate(Adj_all):
        node_degree_global.append(np.zeros(A.shape[0]))
        for u in range(A.shape[0]):
            node_degree_global[rel][u] = np.sum(A[u])

    Adj_all = Adj_all[0]
    print('Adj_all', Adj_all.shape, len(node_degree_global), node_degree_global[0].min(), node_degree_global[0].max())
    time_bar = np.zeros((dataset.N_nodes, 1)) + dataset.FIRST_DATE.timestamp()

    model.initialize(node_embeddings=initial_embeddings,
                        A_initial=Adj_all, keepS=keepS)  # train_loader.dataset.H_train

    return time_bar, node_degree_global

In [ ]:
# Training loop
for epoch in range(epoch_start, epochs + 1):
    # Setting the global time_bar for the datasets
    if not (epoch == epoch_start):
        # Reinitialize node embeddings and adjacency matrices, but keep the model parameters intact
        time_bar, node_degree_global = initalize_state(train_loader.dataset, keepS=epoch > 1)
        model.node_degree_global = node_degree_global
        
    train_loader.dataset.time_bar = time_bar
    test_loader.dataset.time_bar = time_bar

    start = time.time()

    for batch_idx, data_batch in enumerate(train_loader):
        model.train()
        optimizer.zero_grad()

        # Ensure the data is in the correct format
        data_batch[2] = data_batch[2].float()
        data_batch[4] = data_batch[4].double()
        data_batch[5] = data_batch[5].double()

        output = model(data_batch)
        losses = [-torch.sum(torch.log(output[0]) + 1e-10), weight * torch.sum(output[1])]
        if len(output) > 3 and output[-1] is not None:
            losses.extend(output[-1])

        loss = torch.sum(torch.stack(losses)) / batch_size
        loss.backward()
        nn.utils.clip_grad_value_(model.parameters(), 100)
        optimizer.step()

        losses_events.append(losses[0].item())
        losses_nonevents.append(losses[1].item())
        losses_sum.append(loss.item())

        # Clamping psi to prevent numerical overflow
        model.psi.data = torch.clamp(model.psi.data, 1e-1, 1e+3)

        time_iter = time.time() - start

        # Detach computational graph to prevent unwanted backprop
        model.z = model.z.detach()
        model.S = model.S.detach()

        if (batch_idx + 1) % log_interval == 0 or batch_idx == len(train_loader) - 1:
            print(f'\nTRAIN epoch={epoch}/{epochs}, batch={batch_idx + 1}/{len(train_loader)}, '
                  f'sec/iter: {time_iter / (batch_idx + 1):.4f}, loss={loss.item():.3f}, '
                  f'loss components: {[l.item() for l in losses]}')

            # Save state before testing
            variables = get_temporal_variables()
            print('time', datetime.datetime.fromtimestamp(np.max(time_bar)))

            # Testing and collecting results
            result = test(model, n_test_batches=None if batch_idx == len(train_loader) - 1 else 10, epoch=epoch)
            test_MAR.append(np.mean(result[0]['Com']))
            test_HITS10.append(np.mean(result[1]['Com']))
            test_loss.append(result[2])

            # Restore state after testing
            time_bar = set_temporal_variables(variables, model, train_loader, test_loader)

    scheduler.step()